In [2]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [11]:
# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
print('msg:', msg)


# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    keys_list = []
    for _ in range(n):
        x_temp, y_temp = generate_keys(g, p)
        keys_list.append([x_temp, y_temp])
    return keys_list


def concat_and_shuffle_total_keys(keys_list, x, y):
    keys_list.append([x, y])
    random.shuffle(keys_list)
    return keys_list


def get_real_key_position(keys_list, y):
    public_key_list = [key[1] for key in keys_list]
    return public_key_list.index(y)


def get_Y(keys_list):
    public_key_list = [key[1] for key in keys_list]
    Y = ''.join([str(i) for i in public_key_list])
    return Y


def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R


def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    e_list = [None for _ in range(len(keys_list))]
    sig_list = [None for _ in range(len(keys_list))]
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    y_wave = pow(h, x_real, p)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    e_list[idx] = e_initial
    while keys_list[idx][1] != y_real:
        e_last_one = e_list[idx]
        _, random_sig = generate_keys(g, p)
        sig_list[idx] = random_sig
        temp_y = keys_list[idx][1]
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        idx = (idx + 1) % len(total_keys_list)
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    print('s_real:', s_real)
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
        'e_list': e_list
    }
    return total_res
        
    
    


random_keys_count = 10
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
print(total_keys_list)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



msg: Hello, Ring Signature
[[72374070, 604653129], [342596344, 833274426], [148630588, 672546995], [805731059, 709313101], [735676362, 594123532], [3159565, 37822402], [134461049, 989259663], [848872092, 349243916], [881245201, 393421615], [745617738, 252188383], [26305459, 751567341]]
s_real: 278842317


{'e1': 54430018865034553759858670667456336791789864238879435203260013635549331036761,
 'sig_list': [817158969,
  334798594,
  278842317,
  624886493,
  120828532,
  446225387,
  489856807,
  792873835,
  313025621,
  416801398,
  61121530],
 'public_key_list': [604653129,
  833274426,
  672546995,
  709313101,
  594123532,
  37822402,
  989259663,
  349243916,
  393421615,
  252188383,
  751567341],
 'tag': 116075310,
 'msg': 'Hello, Ring Signature',
 'e_list': [54430018865034553759858670667456336791789864238879435203260013635549331036761,
  39378675411975618194911475266739820366622380929799214465075977590722504474701,
  103473853713166396965764771748988761040264690731090554311664916166978429993983,
  42458318145731417045375529481693988018142453844649184452656138123387192883919,
  106788378619280086529380527428026143894638986838967945544210287530181035876892,
  26424922392303282285262821551637134299801543522456382095111212979038345303412,
  961733445218744908543177418327674020576742909

In [10]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 3383467265776348062386361198288888375174019879436322461710818758996229548447
temp_e_V: 72320070539819261577643840437224820880497085340658838032280828081495637467601
temp_e_V: 84274592691073105600249672644395190631340208349991405450163404968286610828485
temp_e_V: 71820076306721944164250636811630893150935383565894876697630448965333726893842
temp_e_V: 80927173031276409488659418912917942145577942890898637425961017093728822150930
temp_e_V: 59587820276916769204080425055433538617756078447161873312312664871894789540610
temp_e_V: 94596725778840186992241938941762522308720620334415852017242420560928130905991
temp_e_V: 83176450916412541625202547045869125367907448488849911398836751052993550283920
temp_e_V: 86030648747737643165767763836404695796204440679811533778344834023819991079969
temp_e_V: 23452189540076770978777852610993794446487327969810652004827843288554514137865
temp_e_V: 52825547179099732634641623674793373134820508852010650410227947438423873183472
338346726577634806238636119828888

In [12]:
msg_1 = 'new test'

total_res = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
total_res

s_real: 991202311


{'e1': 48923946272478011992526161330985508519832922696438091300745419714404005761612,
 'sig_list': [5130156,
  495667344,
  991202311,
  991105033,
  866596237,
  883137621,
  510526722,
  112593422,
  231403890,
  13918913,
  171758160],
 'public_key_list': [604653129,
  833274426,
  672546995,
  709313101,
  594123532,
  37822402,
  989259663,
  349243916,
  393421615,
  252188383,
  751567341],
 'tag': 116075310,
 'msg': 'new test',
 'e_list': [48923946272478011992526161330985508519832922696438091300745419714404005761612,
  84389933241703760147051343608564456386822272093773957847659372006925842442981,
  84284314050017823673406796905255214646198915638242312388649824238211149325348,
  20249911877730973710914464632011955258167269655331968171226690910094086040432,
  50159071508462433306271283137195133823279626048088332001268788897282269268418,
  13370995815112967043505985515808700819916793896687466544224780553296591683385,
  32930950301500554126483046454042491653545773650069034532996258